In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
import semantic_kernel as sk
import os
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

kernel=sk.Kernel()
api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)
chatty = kernel.create_semantic_function("{{ $input }}", temperature=0.0)
    

In [8]:
async def populate_db(kernel: sk.Kernel, df) -> None:
    # Add some documents to the semantic memory
    for index, row in df.iterrows():
        x= row["name"] + " :  " + row["description"]
        await kernel.memory.save_information_async(
        "outdoordb", id=str(index), text=x
    )

In [4]:
import pandas as pd
df = pd.read_csv('OutdoorClothingCatalog_1000.csv')

In [5]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding
kernel.add_text_embedding_generation_service(
        "ada", OpenAITextEmbedding("text-embedding-ada-002", api_key)
    )

In [6]:
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory='catalog'))

In [9]:
await populate_db(kernel,df)

In [10]:
query ="Please suggest a shirt with sunblocking"

In [11]:
query_result1 = await kernel.memory.search_async(collection="outdoordb", limit=1, min_relevance_score=0.3, query=query)

In [12]:
len(query_result1)

1

In [13]:
query_result1[0].text

'Sun Shield Shirt by  :  "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.'